In [1]:
input_file = "mh_refl_bound.out"

import json
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow_probability as tfp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

import sys

sys.path.append("/Users/jay/Desktop/Bachelorarbeit/Implementation")
from dependencies.hbv_sask.model import HBVSASKModel as hbvmodel
from src.execute_model import run_model_single_parameter_node
from src.construct_model import get_model


ndims = 7
dims = ["TT", "C0", "beta", "ETF", "FC", "FRAC", "K2"]
colors = sns.color_palette(n_colors=ndims)

testConfigPath = "/Users/jay/Desktop/Bachelorarbeit/test_config.json"
with open(testConfigPath, "r") as file:
    run_config = json.load(file)

configPath = run_config["configPath"]
basis = run_config["basis"]
model = get_model(configPath, basis)

start_date: 2004-01-01 00:00:00
start_date_predictions: 2005-01-01 00:00:00
end_date: 2006-01-01 00:00:00
simulation length: 365
full_data_range is 732 hours including spin_up_length of 366 hours
simulation_range is of length 366 hours


In [2]:
# Construct params
configurationObject = model.configurationObject
param_lower = []
param_upper = []
for param in configurationObject["parameters"]:
    if param["distribution"] == "Uniform":
        param_lower.append(param["lower"])
        param_upper.append(param["upper"])
    else:
        raise NotImplementedError(
            f"Sorry, the distribution {param['distribution']} is not supported yet"
        )
param_lower = np.array(param_lower)
param_upper = np.array(param_upper)

In [3]:
samples = pd.read_csv(f"{input_file}")
samples

,TT,C0,beta,ETF,FC,FRAC,K2
0,-2.324866,2.989936,1.408014,0.201818,246.702742,0.367610,0.026508
1,-3.805418,2.288647,1.016450,0.310746,280.548676,0.350488,0.036375
2,-2.834961,4.361237,1.686691,0.342479,346.447472,0.614036,0.026062
3,-3.953323,2.929770,1.688850,0.284193,365.135840,0.408091,0.027477
4,-3.281321,3.947231,1.626439,0.303359,232.283334,0.452503,0.015122
...,...,...,...,...,...,...,...
7996,2.397194,6.093376,1.755693,0.562339,338.137724,0.387963,0.042667
7997,2.205853,5.956022,1.734249,0.542937,298.498452,0.288486,0.028221
7998,-0.101905,3.561095,1.860920,0.520007,339.701848,0.193791,0.016727
7999,1.822985,3.990118,1.703870,0.332168,160.039513,0.346584,0.024226


# Plotting

In [4]:
fig = make_subplots(rows=2, cols=4)

# Histogram and KDE
for i, col in enumerate(samples.columns):
    row = (i // 4) + 1
    col_idx = (i % 4) + 1 

    # Histogram
    fig.add_trace(
        go.Histogram(x=samples[col], name=col, histnorm="probability density"),
        row=row,
        col=col_idx,
    )

    # KDE calculation
    kde = gaussian_kde(samples[col])
    x_values = np.linspace(samples[col].min(), samples[col].max(), 300)
    kde_values = kde(x_values)
    fig.add_trace(
        go.Scatter(
            x=x_values, y=kde_values, mode="lines", name=f"KDE {col}", showlegend=False
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=500, 
    width=1200,  
    title_text="Metropolis Hastings with Reflect Boundary: Parameters Overview",
)
fig.show()


In [5]:
fig = make_subplots(rows=2, cols=4)

for i, col in enumerate(samples.columns):
    row = (i // 4) + 1  
    col_idx = (i % 4) + 1  

    fig.add_trace(
        go.Box(
            y=samples[col],
            name=col,
            boxpoints="all",
            jitter=0.5,
            whiskerwidth=0.2,
            marker_size=2,
            line_width=1,
        ),
        row=row,
        col=col_idx,
    )

# Update layout 
fig.update_layout(
    height=500,  
    width=1200,  
    title_text="Metropolis Hastings with Reflect Boundary: Boxplots of Parameters",
)
fig.show()


In [6]:
corr_matrix = samples.corr()

fig = go.Figure(
    data=go.Heatmap(
        z=corr_matrix,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale="Viridis", 
        colorbar=dict(title="Correlation Coefficient"),
    )
)

# Update layout
fig.update_layout(
    title="Metropolis Hastings with Reflect Boundary: Heatmap of Variable Correlations",
    xaxis_title="Variables",
    yaxis_title="Variables",
)

fig.show()

# Sampling

In [7]:
# Sampling Max
param_vec = []
for i in range(len(samples.loc[0])):
    values, counts = np.unique(samples.iloc[:, i], return_counts=True)
    ind = np.argmax(counts)
    param_vec.append(values[ind])

_, posterior_max, _, _ = run_model_single_parameter_node(model, param_vec)

[HVBSASK INFO] [0] parameters: [[-4.544877288023929, -3.647291606509904, 0.9512178732643717, -0.0236338068542441, 50.19004088963214, 0.10024212934969688, -0.0013472259478354424]]


In [8]:
# Mean sampling from posterior
sample_param = []
for i in range(7):
    sample_param.append(np.random.choice(samples.iloc[:, i], 1000))
sample_param = np.array(sample_param).T

posterior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    posterior.append(y_model)

print(np.array(posterior).shape)
posterior_mean = np.mean(np.array(posterior), axis=0)

[HVBSASK INFO] [0] parameters: [array([2.74532747e+00, 8.87790473e+00, 2.04105110e+00, 4.00173627e-01,
       4.04058911e+02, 3.42032153e-01, 2.34653139e-02])]
[HVBSASK INFO] [0] parameters: [array([3.76438107e+00, 1.47357021e+00, 2.01502103e+00, 2.28710232e-01,
       2.77566497e+02, 3.24983526e-01, 2.59385792e-02])]
[HVBSASK INFO] [0] parameters: [array([-3.04344384e-02,  1.33629355e+00,  1.59904187e+00,  5.64026889e-01,
        1.84545970e+02,  5.87672939e-01,  2.94743602e-02])]
[HVBSASK INFO] [0] parameters: [array([-5.99224985e-01,  3.41815210e+00,  2.46607818e+00,  5.69237088e-01,
        2.05054518e+02,  3.81474448e-01,  3.88446990e-02])]
[HVBSASK INFO] [0] parameters: [array([4.61891828e-01, 5.31079130e+00, 1.39936617e+00, 1.77368797e-01,
       3.28960709e+02, 5.85824624e-01, 9.15784410e-03])]
[HVBSASK INFO] [0] parameters: [array([3.81993328e-01, 7.20751592e+00, 2.28330758e+00, 8.18436407e-01,
       4.56793255e+02, 7.95360088e-01, 1.74649312e-02])]
[HVBSASK INFO] [0] paramet

In [9]:
# Mean sampling from prior
sample_param = []
for i in range(7):
    uni = tfp.distributions.Uniform(low=param_lower[i], high=param_upper[i]).sample(1)
    sample_param.append(uni)
sample_param = np.array(sample_param).T

prior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    prior.append(y_model)

_, _, measured_data, _ = run_model_single_parameter_node(model, np.array(vec))

prior_means = np.mean(np.array(prior), axis=0)

2024-06-06 00:14:59.871099: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-06 00:14:59.871125: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-06 00:14:59.871129: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-06 00:14:59.871147: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-06 00:14:59.871159: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[HVBSASK INFO] [0] parameters: [array([3.34167843e+00, 2.42212537e-01, 1.68938453e+00, 9.15461363e-01,
       4.41517017e+02, 4.27633169e-01, 8.57055374e-03])]
[HVBSASK INFO] [0] parameters: [array([3.34167843e+00, 2.42212537e-01, 1.68938453e+00, 9.15461363e-01,
       4.41517017e+02, 4.27633169e-01, 8.57055374e-03])]


In [10]:
fig = go.Figure()
model.get_start_date
dates = model.get_simulation_range()
fig.add_trace(
    go.Scatter(
        x=dates,
        y=prior_means,
        mode="lines",
        name="Prior Mean",
        line=dict(color="lightgrey"),
    )
)
fig.add_trace(
    go.Scatter(x=dates, y=posterior_mean, mode="lines", name="Posterior Mean")
)
fig.add_trace(go.Scatter(x=dates, y=posterior_max, mode="lines", name="Posterior Max"))
fig.add_trace(go.Scatter(x=dates, y=measured_data, mode="lines", name="Measured Data"))

# Update layout
fig.update_layout(
    title="Metropolis Hastings with Reflect Boundary: Bayesian Inference Result Comparison",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Time Series",
    hovermode="x unified",
    template="plotly_white",
)

fig.show()

In [11]:
def rmse(result, target):
    diff = result - target
    aggr = 0
    for i in range(len(diff)):
        aggr += diff[i] ** 2
    rmse = (aggr / (len(diff))) ** 0.5
    return rmse


def mae(result, target):
    return np.absolute(result - target).mean()


print(f"RMSE of Posterior Mean: {rmse(posterior_mean, measured_data)}")
print(f"RMSE of Posterior Max: {rmse(posterior_max, measured_data)}")
print(f"MAE of Posterior Mean: {mae(posterior_mean, measured_data)}")
print(f"MAE of Posterior Max: {mae(posterior_max, measured_data)}")

RMSE of Posterior Mean: 23.049245979474453
RMSE of Posterior Max: 69.71205332181275
MAE of Posterior Mean: 12.112286983423026
MAE of Posterior Max: 65.60333165723709
